In [1]:
using BenchmarkTools

Compile it with ifort, and check the exported symbols:
```
call "c:\Program Files (x86)\IntelSWTools\compilers_and_libraries_2019.5.281\windows\bin\ifortvars.bat" intel64
ifort -O2 /dll clipping.f90
DUMPBIN /exports clipping.dll
```

In [2]:
function area_of_intersection(a, b)
    ndim, nvertex, ntriangles = size(a)
    out = Vector{Float64}(undef, ntriangles)
    ccall(
        (:SUTHERLAND_HODGMAN_mp_AREA_OF_INTERSECTION, "./src/fortran/clipping.dll"),
        Nothing,
        (Ptr{Int64}, Ptr{Int64}, Ptr{Int64}, Ptr{Float64}, Ptr{Float64}, Ptr{Float64}),
        Ref(ndim), Ref(nvertex), Ref(ntriangles), a, b, out,
    )
    return out
end

area_of_intersection (generic function with 1 method)

In [3]:
function ccw!(a)
    n = size(a)[end]
    
    for i in 1:n
        t = a[:, :, i]
        normal = (t[1, 2] - t[1, 1])*(t[2, 3]-t[2, 1])-(t[2, 2]-t[2, 1])*(t[1, 3]-t[1, 1])

        if normal < 0
            a[:, :, i] .= reverse(t, dims=2)
        end         
    end
end

a = rand(2, 3, 1000000)
b = rand(2, 3, 1000000)
ccw!(a)
ccw!(b)

In [4]:
@btime area_of_intersection(a, b)

  109.394 ms (2 allocations: 7.63 MiB)


1000000-element Array{Float64,1}:
 0.0                  
 0.0                  
 0.0                  
 0.006465156978919396 
 0.0                  
 0.0                  
 0.0                  
 0.0                  
 0.002674764922565896 
 0.0                  
 0.008159821809748776 
 0.0041256631173297595
 0.08427987663436645  
 ⋮                    
 0.025755665204455085 
 0.029806514418248577 
 0.0                  
 0.0005411828441671707
 0.03094450651451533  
 0.0                  
 4.6029137721000114e-5
 0.0                  
 0.0022481726737602495
 0.011791581751828402 
 0.050894814005172595 
 0.004642206136832096 